In [ ]:
DATA_url = r"http://thredds.northwestknowledge.net:8080/thredds/dodsC/agg_met_pr_1979_CurrentYear_CONUS.nc"


In [ ]:
import os
import logging
import xarray as xr
logging.basicConfig(level=logging.INFO, force=True)


In [ ]:
%run ../utils.ipynb
_versions(['xarray'])

In [ ]:
# lazy-load
ds_in = xr.open_dataset(DATA_url + r"#fillmismatch", decode_coords=True, chunks={})#, decode_times=False)
# and show it:
ds_in

In [ ]:
ds_in.precipitation_amount

In [ ]:
16169/61

In [ ]:
ds_in = xr.open_dataset(
    DATA_url + r"#fillmismatch", 
    decode_coords=True, 
    chunks={'time': 24, 'lon': 864, 'lat': 432}
)
ds_in

In [ ]:
8640/1440

In [ ]:
4320/720